## Data Generation

In [ ]:
import numpy as np
from scipy import spatial

In [ ]:
def generate_class_vectors(k, d):
    W = np.ones([k, d])
    # Generate k vectors for classes
    mean = [0] * d
    cov = np.eye(d)
    for j in range(0, k):
        vec = np.random.multivariate_normal(mean, cov)
        vec = vec / np.linalg.norm(vec)
        W[j, ] = vec

    return W

In [ ]:
def generate_argmax_data(k, d, num_samples):
    W = generate_class_vectors(k, d)

    X = np.zeros([num_samples, d])
    Y = np.zeros([num_samples, 1])

    mean = [0] * d
    cov = np.eye(d)
    for i in range(0, num_samples):
        vec = np.random.multivariate_normal(mean, cov)
        vec = vec / np.linalg.norm(vec)
        X[i, ] = vec

        true_y = get_closest_vector(W, vec, k)
        r = np.random.uniform()
        if(r < 0.95):
            Y[i, ] = true_y
        else:
            Y[i, ] = np.random.randint(0, k)

    return (W, X, Y)

In [ ]:
k = 100
n = 10000
d = 3
(W, X, Y) = generate_argmax_data(k, d, n)

In [ ]:
np.savetxt('multiclass_train_features_k' + str(k) + 'n' + str(n) + '.csv', X[1:int(n/2), :], delimiter=',')
np.savetxt('multiclass_train_labels_k' + str(k) + 'n' + str(n) + '.csv', Y[1:int(n/2), :], delimiter=',')
np.savetxt('multiclass_test_features_k' + str(k) + 'n' + str(n) + '.csv', X[int(n/2):, :], delimiter=',')
np.savetxt('multiclass_test_labels_k' + str(k) + 'n' + str(n) + '.csv', Y[int(n/2):, :], delimiter=',')

In [ ]:
def generate_softmax_data(k, d, num_samples):
    W = generate_class_vectors(k, d)

    X = np.zeros([num_samples, d])
    Y = np.zeros([num_samples, 1])

    cov = np.eye(d) * (get_lowest_vector_distance(W) / (1000 * k))
    
    i = 0
    while(i < num_samples):
        true_y = np.random.randint(0, k)

        vec = np.random.multivariate_normal(W[true_y, ], cov)
        vec = vec / np.linalg.norm(vec)
        Y_dist = np.exp(np.matmul(W, vec))
        Y_dist = Y_dist / np.sum(Y_dist)
        if(log_condition(Y_dist, k)) :
            label = np.random.choice(np.arange(0, k), p=Y_dist)
            Y[i, ] = label
            X[i, ] = vec
            i = i + 1
        

    return (W, X, Y)

In [ ]:
def log_condition(Y_dist, k):
    largest = np.amax(Y_dist)
    if(Y_dist[~(Y_dist == largest)].size == 0):
        return False
    second_largest = np.amax(Y_dist[~(Y_dist == largest)])
    
    # Slight change to condition to allow for more 
    return largest > ( second_largest + 1 / np.log(k))

In [ ]:
log_condition(np.array([0, 1]), 50)

In [ ]:
def get_lowest_vector_distance(W):
    arr = spatial.distance.cdist(W, W)
    return np.min(arr[np.nonzero(arr)])


def get_closest_vector(W, vec, k):
    best_distance = float('inf')
    for i in range(0, k):
        dist = np.linalg.norm(W[i, ] - vec)
        if(dist < best_distance):
            best_distance = dist
            y = i
    return y

### Generate the data for Testing

In [ ]:
(W, X, Y) = generate_argmax_data(12, 10, 100)

# Learn Classifiers

In [ ]:
from scipy import optimize, stats

In [ ]:
## Returns the ith linear classifier using gradient descent
def learn_linear_classifier_grad_descent(i, s, X, Y, norm):
    d = X.shape[1]
    eta = 0.001
    precision = 10**(-1)
    max_iter = 10**5
    
    iters = 0
    w_hat = np.ones([1, d]) * (i + 1)/d
    curr_opt = opt_function(w_hat, args=(i, s, X, Y))
    while((iters < max_iter) and (iters == 0 or (curr_opt >= precision))):
        prev_opt = curr_opt
        prev_w_hat = w_hat
        
        w_hat = prev_w_hat - eta * obj_gradient(i, s, prev_w_hat, X, Y, norm)
        
        iters = iters + 1
        curr_opt = opt_function(w_hat, args=(i, s, X, Y))
        ## eta = (100 + iters)**(-0.75)
    
    print('Num Iterations: ' + str(iters))
    print('Obj Function Val: ' + str(opt_function(w_hat, args=(i, s, X, Y))))
    print('Final Vector:' + str(w_hat))
    return w_hat

In [ ]:
norm = 1
w = learn_linear_classifier_grad_descent(1, 2, X_test, Y_test, norm)

In [ ]:
X_test = np.random.rand(500, 2)
Y_test = np.zeros([500, 1])
for i in range(0, 500):
    if(X_test[i, 0] > X_test[i, 1]):
        Y_test[i, 0] = 1

In [ ]:
estimates = np.matmul(X_test, np.transpose(w))
estimates

In [ ]:
Y_test

In [ ]:
np.sum(np.abs(np.round(estimates) - Y_test)) / 500

In [ ]:
W_hat = learn_all_classifiers(5, X_test, Y_test, 1)

In [ ]:
sample = 14
print(X_test[sample, ])
print(Y_test[sample, ])
print(classify_sample(W_hat, X_test[sample, ]))

In [ ]:
loss(W_hat, X_test, Y_test)

In [ ]:
def obj_gradient(i, s, w, X, Y, norm):
    estimates = np.matmul(X, np.transpose(w))
    error = estimates - Y
    pos_gradient = X * (1 - i / s) * (error > 0)
    neg_gradient = X * (i / s) * (error < 0)
    pos_gradient = np.mean(pos_gradient, axis=0)
    neg_gradient = np.mean(neg_gradient, axis=0)
    
    lmda = 0.01
    reg = w * 0
    if(norm == 1):
        reg = np.abs(w) / w
    elif(norm == 2):
        reg = 2 * w 
    return pos_gradient - neg_gradient + lmda * reg

In [ ]:
## Returns the ith linear classifier
def learn_linear_classifier(i, s, X, Y, norm):
    d = X.shape[1]
    w_hat = optimize.minimize(opt_function, [1/d]*d, args=(i, s, X, Y, norm), method='Nelder-Mead').x
    return w_hat

In [ ]:
def opt_function(w_hat, args):
    i = args[0]
    s = args[1]
    X = args[2]
    Y = args[3]
    estimates = np.matmul(X, np.transpose(w_hat))
    error = estimates - Y
    pos_error = error * (1 - i / s) * (error > 0)
    neg_error = -error * (i / s) * (error < 0)
    pos_mean = np.mean(pos_error, axis=0)
    neg_mean = np.mean(neg_error, axis=0)
    return pos_mean + neg_mean

In [ ]:
def learn_all_classifiers(s, X, Y, norm):
    d = X.shape[1]
    W_hat = np.zeros([s, d])

    for i in range(0, s):
        W_hat[i, ] = learn_linear_classifier_grad_descent(i, s, X, Y, norm)

    return W_hat

In [ ]:
def classify_sample(W_hat, x):
    quantiles = np.matmul(W_hat, x)
    print(quantiles)
    quantiles = np.round(quantiles)
    return stats.mode(quantiles)[0][0]

In [ ]:
W_hat

# Tests

In [ ]:
def loss(W_hat, X, Y):
    num_samples = X.shape[0]
    total = 0
    for i in range(0, num_samples):
        if(classify_sample(W_hat, X[i, ]) != Y[i,]):
            total += 1
    return total / num_samples

In [ ]:
def output(W_hat, X, Y):
    num_samples = X.shape[0]
    total = 0
    for i in range(0, num_samples):
        print(classify_sample(W_hat, X[i, ]))
        print(Y[i, ])

### Single Test

In [ ]:
num_samples = 1000
k = 32 # For k large enough (> 30 suffices), argmax data satisfies condition
d = int(3 * np.log2(k))
s = int(3 * np.log2(k))
norm = 1

In [ ]:
(W, X, Y) = generate_argmax_data(k, d, num_samples)

In [ ]:
print(W)

In [ ]:
%%time
W_hat = learn_all_classifiers(s, X, Y, norm)

In [ ]:
W_hat

In [ ]:
loss(W_hat, X, Y)

In [ ]:
sample = 50
print(X[sample, ])
print(Y[sample, ])
print(classify_sample(W_hat, X[sample, ]))

In [ ]:
## Logistic Regression for comparison
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X, Y.flatten())

In [ ]:
sum(clf.predict(X) != Y.flatten()) / num_samples

In [ ]:
## OVA SVM
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(X, Y.flatten()) 

In [ ]:
sum(lin_clf.predict(X) != Y.flatten()) / num_samples

### Other Test

In [ ]:
num_samples = 1000
k = 100
d = int(10 * np.log10(k))
s = int(10 * np.log10(k))
norm = 1